# PrimeurVision — Fine-tuning YOLOv8

Fine-tuning d'un modèle **YOLOv8n pré-entraîné sur COCO** pour détecter 6 classes de fruits et légumes en contexte culinaire.

**Stratégie d'entraînement en 2 phases :**
1. **Backbone gelé** : seule la tête de détection est entraînée — on adapte rapidement le modèle à nos classes sans perturber les features générales apprises sur COCO.
2. **Fine-tuning complet** : on dégèle tout le réseau et on affine avec un learning rate plus faible — permet d'optimiser finement toutes les couches sur notre domaine.

**Classes** : carotte (0), aubergine (1), citron (2), pomme_de_terre (3), radis (4), tomate (5)

**Dataset** : 238 images — 166 train / 36 val / 36 test

## 1. Installation des dépendances

In [ ]:
!pip install ultralytics -q

## 2. Imports

In [ ]:
import os
import shutil
import random
import glob
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from ultralytics import YOLO

## 3. Hyperparamètres

| Paramètre | Phase 1 | Phase 2 | Justification |
|---|---|---|---|
| Epochs | 10 | 40 | Phase 1 courte : convergence rapide de la tête. Phase 2 plus longue : affinage fin |
| Learning rate | 1e-2 | 1e-3 | LR élevé en phase 1 pour adapter vite la tête ; LR faible en phase 2 pour ne pas écraser les features |
| Freeze | 10 couches | 0 | On gèle le backbone (couches 0–9) en phase 1, puis on libère tout |
| Batch size | 16 | 16 | Compromis mémoire/stabilité des gradients sur Colab GPU |
| Image size | 640 | 640 | Taille standard YOLOv8, bon compromis précision/vitesse |
| Patience | 10 | 15 | Early stopping : arrêt si la mAP@50 ne s'améliore pas pendant N epochs |

In [ ]:
# --- HYPERPARAMÈTRES ---

# Phase 1 : backbone gelé
PHASE1_EPOCHS  = 10
PHASE1_LR      = 1e-2
FREEZE_LAYERS  = 10

# Phase 2 : fine-tuning complet
PHASE2_EPOCHS  = 40
PHASE2_LR      = 1e-3

# Paramètres communs
IMG_SIZE       = 640
BATCH_SIZE     = 16
PATIENCE_P1    = 10
PATIENCE_P2    = 15
CONF_THRESHOLD = 0.25

## 4. Montage Google Drive et chargement du dataset

Uploadez le dossier `dataset/` sur Google Drive dans `My Drive/PrimeurVision/dataset/`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATASET_SRC = '/content/drive/MyDrive/PrimeurVision/dataset'
WORK_DIR    = '/content/dataset'

# Copie en local pour accélérer l'entraînement
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)
shutil.copytree(DATASET_SRC, WORK_DIR)

# Charger et mettre à jour la config avec les chemins locaux
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

data_config['path']  = WORK_DIR
data_config['train'] = 'images/train'
data_config['val']   = 'images/val'
data_config['test']  = 'images/test'
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

CLASS_NAMES = data_config['names']
NUM_CLASSES = len(CLASS_NAMES)

print(f"{NUM_CLASSES} classes : {list(CLASS_NAMES.values())}")
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(WORK_DIR, 'images', split)
    if os.path.exists(img_dir):
        print(f"  {split}: {len(os.listdir(img_dir))} images")

## 5. Exploration du dataset

In [ ]:
COLORS = ['#FF8C00', '#9B59B6', '#FFD700', '#8B4513', '#E74C3C', '#FF4444']

def parse_yolo_label(label_path):
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])
                annotations.append((class_id, x_center, y_center, width, height))
    return annotations

def plot_image_with_boxes(img_path, label_path, ax):
    img = Image.open(img_path)
    w, h = img.size
    ax.imshow(img)
    for class_id, xc, yc, bw, bh in parse_yolo_label(label_path):
        x1 = (xc - bw / 2) * w
        y1 = (yc - bh / 2) * h
        color = COLORS[class_id % len(COLORS)]
        rect = patches.Rectangle((x1, y1), bw * w, bh * h,
                                  linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1 - 5, CLASS_NAMES.get(class_id, '?'), color=color,
                fontsize=9, fontweight='bold', backgroundcolor='black')
    ax.axis('off')

# 6 images aléatoires du train avec leurs bounding boxes
train_images_dir = os.path.join(WORK_DIR, 'images', 'train')
train_labels_dir = os.path.join(WORK_DIR, 'labels', 'train')
image_files   = sorted(glob.glob(os.path.join(train_images_dir, '*.jpg')))
sample_images = random.sample(image_files, min(6, len(image_files)))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, img_path in zip(axes.flatten(), sample_images):
    base_name  = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(train_labels_dir, base_name + '.txt')
    if os.path.exists(label_path):
        plot_image_with_boxes(img_path, label_path, ax)
plt.suptitle('Échantillon du dataset (train) avec annotations', fontsize=14)
plt.tight_layout()
plt.show()

# Distribution des classes (nb images par classe par split)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, split in zip(axes, ['train', 'val', 'test']):
    labels_dir   = os.path.join(WORK_DIR, 'labels', split)
    class_counts = {v: 0 for v in CLASS_NAMES.values()}
    for lf in glob.glob(os.path.join(labels_dir, '*.txt')):
        seen = set()
        for class_id, *_ in parse_yolo_label(lf):
            name = CLASS_NAMES.get(class_id)
            if name and name not in seen:
                class_counts[name] += 1
                seen.add(name)
    bars = ax.bar(class_counts.keys(), class_counts.values(), color=COLORS)
    ax.bar_label(bars)
    ax.set_title(f'{split} ({sum(class_counts.values())} images)')
    ax.set_ylim(0, max(class_counts.values()) + 5)
    ax.tick_params(axis='x', rotation=30)
plt.suptitle('Distribution des classes par split', fontsize=13)
plt.tight_layout()
plt.show()

## 6. Entraînement en 2 phases

### Phase 1 — Backbone gelé

On charge YOLOv8n pré-entraîné sur COCO et on gèle les 10 premières couches (backbone CSP-DarkNet). Seule la tête de détection apprend. Cette approche permet :
- D'éviter le *catastrophic forgetting* des features génériques (bords, textures, formes)
- D'obtenir une convergence rapide sur nos classes cibles
- De réduire le risque d'overfitting avec notre petit dataset (~166 images train)

In [ ]:
model = YOLO('yolov8n.pt')

results_phase1 = model.train(
    data=data_yaml_path,
    epochs=PHASE1_EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    lr0=PHASE1_LR,
    freeze=FREEZE_LAYERS,
    name='primeurvision_phase1',
    patience=PATIENCE_P1,
    save=True,
    plots=True,
    verbose=True
)

### Phase 2 — Fine-tuning complet

On reprend le meilleur modèle de la phase 1 et on dégèle toutes les couches. Le learning rate est réduit d'un facteur 10 pour affiner les poids sans écraser les représentations apprises lors de la phase 1.

In [ ]:
best_phase1 = os.path.join('runs', 'detect', 'primeurvision_phase1', 'weights', 'best.pt')
model = YOLO(best_phase1)

results_phase2 = model.train(
    data=data_yaml_path,
    epochs=PHASE2_EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    lr0=PHASE2_LR,
    freeze=0,
    name='primeurvision_phase2',
    patience=PATIENCE_P2,
    save=True,
    plots=True,
    verbose=True
)

RESULTS_DIR = os.path.join('runs', 'detect', 'primeurvision_phase2')

## 7. Courbes d'entraînement et résultats sur la validation

In [ ]:
# Courbes loss + mAP au fil des epochs (phase 2)
results_img = os.path.join(RESULTS_DIR, 'results.png')
if os.path.exists(results_img):
    plt.figure(figsize=(18, 8))
    plt.imshow(Image.open(results_img))
    plt.axis('off')
    plt.title("Courbes d'entraînement — Phase 2 (fine-tuning complet)")
    plt.show()

# Matrice de confusion normalisée (validation)
for fname in ['confusion_matrix_normalized.png', 'confusion_matrix.png']:
    confusion_img = os.path.join(RESULTS_DIR, fname)
    if os.path.exists(confusion_img):
        plt.figure(figsize=(8, 8))
        plt.imshow(Image.open(confusion_img))
        plt.axis('off')
        plt.title('Matrice de confusion — validation (phase 2)')
        plt.show()
        break

# Courbe Précision-Rappel
pr_img = os.path.join(RESULTS_DIR, 'PR_curve.png')
if os.path.exists(pr_img):
    plt.figure(figsize=(10, 6))
    plt.imshow(Image.open(pr_img))
    plt.axis('off')
    plt.title('Courbe Précision-Rappel par classe')
    plt.show()

# Métriques finales sur la validation
metrics_val = model.val(data=data_yaml_path, split='val')
print("\n=== Résultats sur la validation (phase 2) ===")
print(f"  mAP@50    : {metrics_val.box.map50:.4f}")
print(f"  mAP@50-95 : {metrics_val.box.map:.4f}")
print(f"  Précision : {metrics_val.box.mp:.4f}")
print(f"  Recall    : {metrics_val.box.mr:.4f}")
print("\nAP@50 par classe :")
for i, name in CLASS_NAMES.items():
    ap = metrics_val.box.ap50[i] if i < len(metrics_val.box.ap50) else 0
    print(f"  {name:20s} : {ap:.4f}")

## 8. Sauvegarde du modèle sur Drive

In [ ]:
save_dir = '/content/drive/MyDrive/PrimeurVision/models'
os.makedirs(save_dir, exist_ok=True)

# Meilleur modèle (phase 2)
best_model_path = os.path.join(RESULTS_DIR, 'weights', 'best.pt')
shutil.copy2(best_model_path, os.path.join(save_dir, 'best_yolov8n_primeurvision.pt'))

# Courbes, matrices et figures utiles pour le rapport
for fname in ['results.png', 'confusion_matrix.png', 'confusion_matrix_normalized.png',
              'PR_curve.png', 'F1_curve.png']:
    src = os.path.join(RESULTS_DIR, fname)
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(save_dir, fname))

print(f"Modèle et courbes sauvegardés dans : {save_dir}")
print(f"  → best_yolov8n_primeurvision.pt")